In [ ]:
# 📦 Install dependencies (first time only)
!pip install -q langchain faiss-cpu pypdf langchain-community
!pip install -q sentence-transformers transformers bs4

# 📥 Step 1: Load and Chunk Web Data
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = WebBaseLoader(
    web_paths=("https://ai.pydantic.dev/",),
)
docs = loader.load()
print(f"✅ Loaded {len(docs)} documents")

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
chunks = splitter.split_documents(docs)
print(f"✅ Split into {len(chunks)} chunks")


✅ Loaded 1 documents
✅ Split into 18 chunks


In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(chunks, embedding_model)
print("✅ FAISS Vectorstore created")


✅ FAISS Vectorstore created


In [ ]:
from transformers import pipeline

# Local QA model (downloads once, then cached)
qa_pipeline = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")
print("✅ Local QA pipeline ready")


Device set to use cpu


✅ Local QA pipeline ready


In [ ]:
def local_rag_qa(query, top_k=3):
    # Search FAISS for relevant chunks
    docs = vectorstore.similarity_search(query, k=top_k)
    context = "\n".join([doc.page_content for doc in docs])

    # Run QA over combined context
    result = qa_pipeline({
        "question": query,
        "context": context
    })

    print(f"\n❓ Question: {query}")
    print(f"📎 Answer: {result['answer']}\n")
    print("📚 Top Contexts:")
    for doc in docs:
        print("-", doc.page_content[:150], "...\n")


In [ ]:
questions = [
    "What is PydanticAI and why was it created?",
    "How is PydanticAI similar to FastAPI in terms of developer experience?",
    "What makes PydanticAI type-safe and structured?",
    "How does PydanticAI support streaming and debugging?",
    "What is llms.txt?"
]

for q in questions:
    local_rag_qa(q)


/usr/local/lib/python3.11/dist-packages/transformers/pipelines/question_answering.py:390: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(



❓ Question: What is PydanticAI and why was it created?
📎 Answer: to make it less painful to
  build production grade applications with Generative AI

📚 Top Contexts:
- PydanticAI




















 






          Skip to content
        















            PydanticAI
          



            
            ...

- This Pydantic model is used to constrain the structured data returned by the agent. From this simple definition, Pydantic builds the JSON Schema that  ...

- pydantic_graph.exceptions
    
  





    pydantic_evals.dataset
    
  





    pydantic_evals.evaluators
    
  





    pydantic_evals.reporting ...



/usr/local/lib/python3.11/dist-packages/transformers/pipelines/question_answering.py:390: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(



❓ Question: How is PydanticAI similar to FastAPI in terms of developer experience?
📎 Answer: Python agent framework

📚 Top Contexts:
- FastAPI revolutionized web development by offering an innovative and ergonomic design, built on the foundation of Pydantic.
Similarly, virtually every ...

- pydantic_graph.exceptions
    
  





    pydantic_evals.dataset
    
  





    pydantic_evals.evaluators
    
  





    pydantic_evals.reporting ...

- PydanticAI




















 






          Skip to content
        















            PydanticAI
          



            
            ...



/usr/local/lib/python3.11/dist-packages/transformers/pipelines/question_answering.py:390: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(



❓ Question: What makes PydanticAI type-safe and structured?
📎 Answer: to make it less painful to
  build production grade applications with Generative AI

📚 Top Contexts:
- This Pydantic model is used to constrain the structured data returned by the agent. From this simple definition, Pydantic builds the JSON Schema that  ...

- PydanticAI




















 






          Skip to content
        















            PydanticAI
          



            
            ...

- pydantic_graph.exceptions
    
  





    pydantic_evals.dataset
    
  





    pydantic_evals.evaluators
    
  





    pydantic_evals.reporting ...


❓ Question: How does PydanticAI support streaming and debugging?
📎 Answer: integrates with Pydantic Logfire

📚 Top Contexts:
- FastAPI revolutionized web development by offering an innovative and ergonomic design, built on the foundation of Pydantic.
Similarly, virtually every ...

- pydantic_graph.exceptions
    
  





    pydantic_evals.dataset
  

/usr/local/lib/python3.11/dist-packages/transformers/pipelines/question_answering.py:390: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(



❓ Question: What is llms.txt?
📎 Answer: a file containing a brief description
  of the project

📚 Top Contexts:
- As of today, these files cannot be natively leveraged by LLM frameworks or IDEs. Alternatively,
an MCP server can be implemented to properly parse the ...

- Configure logfire, this will fail if project is not set up.
In our demo, DatabaseConn uses asyncpg to connect to a PostgreSQL database, so logfire.ins ...

- Dynamic system prompts can be registered with the @agent.system_prompt decorator, and can make use of dependency injection. Dependencies are carried v ...

